In [7]:
import pandas as pd
from unidecode import unidecode
paths_outputs = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Indicadores/terceiros/"
paths_inputs = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/"

In [84]:
path =  "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Dicionario, Documentanção e Codigos/dicionario_regioes.csv"

,cod_uf,nome_uf,cod_regiao_geo_intermediaria,nome_regiao_geo_intermediaria,cod_regiao_geo_imediata,nome_regiao_geo_imediata,cod_mesoregiao_geo,nome_mesoregiao_geo,cod_microregiao,nome_microregiao,nome_municipio,cod_municipio,sigla_uf,nome_regiao,censo_2010,capital
0,11,Rondônia,1101,Porto Velho,110001,Porto Velho,1,Madeira-Guaporé,1,Porto Velho,Candeias do Jamari,1100809,RO,Norte,19779.0,Não
1,11,Rondônia,1101,Porto Velho,110001,Porto Velho,1,Madeira-Guaporé,2,Guajará-Mirim,Guajará-Mirim,1100106,RO,Norte,41656.0,Não
2,11,Rondônia,1101,Porto Velho,110001,Porto Velho,1,Madeira-Guaporé,1,Porto Velho,Itapuã do Oeste,1101104,RO,Norte,8566.0,Não
3,11,Rondônia,1101,Porto Velho,110001,Porto Velho,1,Madeira-Guaporé,1,Porto Velho,Nova Mamoré,1100338,RO,Norte,22546.0,Não
4,11,Rondônia,1101,Porto Velho,110001,Porto Velho,1,Madeira-Guaporé,1,Porto Velho,Porto Velho,1100205,RO,Norte,428527.0,Capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,35,São Paulo,3501,São Paulo,350002,Santos,15,Metropolitana de São Paulo,63,Santos,Cubatão,3513504,SP,Sudeste,118720.0,Não
5566,35,São Paulo,3501,São Paulo,350002,Santos,15,Metropolitana de São Paulo,63,Santos,Guarujá,3518701,SP,Sudeste,290752.0,Não
5567,35,São Paulo,3501,São Paulo,350002,Santos,15,Metropolitana de São Paulo,63,Santos,Praia Grande,3541000,SP,Sudeste,262051.0,Não
5568,35,São Paulo,3501,São Paulo,350002,Santos,15,Metropolitana de São Paulo,63,Santos,Santos,3548500,SP,Sudeste,419400.0,Não


In [3]:
nascidos_vivos = pd.read_parquet(paths_inputs+"nascidos_vivos_tratados.parquet")
nascidos_vivos = nascidos_vivos[["cod_municipio", "contagem"]]
nascidos_vivos = nascidos_vivos.rename(columns={"contagem": "nascidos_vivos"})

populacao = pd.read_parquet(paths_inputs+"populacao_tratados.parquet")
populacao = populacao[["cod_municipio", "pop_2021"]]

#### Eleitorado

In [170]:
path_pref =  "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Oportunidades/eleitorado/eleitorado/municipio_prefeito_2020.csv"
path_vereador =  "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Oportunidades/eleitorado/eleitorado/municipio_vereador_2020.csv"

df_pref = pd.read_csv(path_pref, encoding="latin-1", sep =";")
df_pref = df_pref[["sg_uf", "nm_ue", "sg_partido", "qt_cand_eleitos"]]

df_vereador = pd.read_csv(path_vereador, encoding="latin-1", sep =";")
df_vereador = df_vereador[["sg_uf", "nm_ue", "sg_partido", "qt_cand_eleitos"]]

In [140]:
df_pref = df_pref.rename(
    columns={"nm_ue":"nome_municipio",
             "sg_partido":"partido",
             "qt_cand_eleitos":"eleitos",
             'sg_uf':'sigla_uf'})

df_pref["eleitos"] = df_pref["eleitos"].apply(lambda x: x.split(",")[0])

df_pref["eleitos"] = df_pref["eleitos"].astype(int).clip(upper=1)

df_pref["nome_municipio"] = df_pref["nome_municipio"].str.lower().map(unidecode)

In [141]:
df_vereador = df_vereador.rename(
    columns={"nm_ue":"nome_municipio",
             "sg_partido":"partido",
             "qt_cand_eleitos":"eleitos",
             'sg_uf':'sigla_uf'})
df_vereador["eleitos"] = df_vereador["eleitos"].apply(lambda x: x.split(",")[0])

df_vereador["eleitos"] = df_vereador["eleitos"].astype(int).clip(upper=1)

df_vereador["nome_municipio"] = df_vereador["nome_municipio"].str.lower().map(unidecode)

In [142]:
eleitos_geral = pd.concat([df_pref, df_vereador])
eleitos_geral.drop_duplicates(inplace=True)

In [143]:
eleitos_geral_total = eleitos_geral.groupby(["sigla_uf", "nome_municipio"], as_index=False).count()[["sigla_uf", "nome_municipio", "partido"]]
eleitos_geral_total = eleitos_geral_total.rename(columns={"partido":"n_partidos"})

In [158]:
dicio_new = {
    "graccho cardoso":"gracho cardoso",
    "alvorada do oeste":"alvorada d'oeste",
    "espigao do oeste":"espigao d'oeste",
    "olhos d'agua":"olhos-d'agua",
    "santa isabel do para":"santa izabel do para",
    "amparo de sao francisco":"amparo do sao francisco",
    "arez":"ares",
    "santo antonio do leverger":"santo antonio de leverger",
    "eldorado dos carajas":"eldorado do carajas",
    "camaca":"camacan",
    "pingo d'agua":"pingo-d'agua",
    "sao thome das letras":"sao tome das letras",
    "sem peixe":"sem-peixe",
    "dona eusebia":"dona euzebia",
    "sao luis do paraitinga":"sao luiz do paraitinga",
    "assu":"acu",
    "boa saude":"januario cicco"
    }

In [159]:
eleitos_geral_vitoriosos = eleitos_geral.query("eleitos == 1")

eleitos_geral_vitoriosos = eleitos_geral_vitoriosos.groupby(["sigla_uf", "nome_municipio"], as_index=False).count()[["sigla_uf", "nome_municipio", "partido"]]
eleitos_geral_vitoriosos = eleitos_geral_vitoriosos.rename(columns={"partido":"n_partidos_vitoriosos"})


In [160]:
dicio["nome_municipio"] = dicio["nome_municipio"].str.lower().map(unidecode)
porcentagem_eleitos = eleitos_geral_vitoriosos.merge(eleitos_geral_total, on=["sigla_uf", "nome_municipio"], how="left")
porcentagem_eleitos["porcentagem_eleitos"] = porcentagem_eleitos["n_partidos_vitoriosos"]/porcentagem_eleitos["n_partidos"]
porcentagem_eleitos = porcentagem_eleitos.replace(dicio_new, regex=False)

In [166]:
merged = dicio.merge(porcentagem_eleitos, on=["sigla_uf", "nome_municipio"], how="outer").sort_values("n_partidos_vitoriosos")

In [167]:
merged = merged[["sigla_uf", "nome_municipio2", "n_partidos_vitoriosos", 'n_partidos', 'porcentagem_eleitos']]

In [169]:
merged.to_csv(paths_outputs + "proporcao_eleitos.csv", index=False)

#### Frotas Onibus e Micro-onibus

In [4]:
dfs = []
for i in range (3):
    path = f"C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Oportunidades/frotas/frotas_frotas00000000000{i}.parquet"
    df = pd.read_parquet(path)
    dfs.append(df)
data = pd.concat(dfs)


In [8]:
pd.set_option('display.max_columns', None)

In [17]:
data = data[["id_municipio", "ano", "mes", "microonibus", "onibus"]]
data = data.query("ano == 2021")
data["frota_onibus_microonibus"] = data["microonibus"] + data["onibus"]
data = data[["id_municipio", "frota_onibus_microonibus"]].groupby("id_municipio", as_index=False).mean()
data = data.rename(columns={"id_municipio":"cod_municipio"})
data["frota_onibus_microonibus"] = data["frota_onibus_microonibus"].astype(int)

In [24]:
frota_onibus = data.merge(populacao, on="cod_municipio", how="left")
frota_onibus["frota_onibus_microonibus_por_1k_hab"] = frota_onibus["frota_onibus_microonibus"]*1000/frota_onibus["pop_2021"]

In [26]:
frota_onibus.to_csv(paths_outputs + "frota_onibus_microonibus.csv", index=False)

#### Eventos

#### RAIS

###### RAIS Ensino Superior

In [34]:
path = 'C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/rais_tratada/rais_ensino_superior/'
rais_estudos_2021 = pd.read_parquet(path+"rais_ensino_superior000000000017.parquet")

rais_estudos_2021 = rais_estudos_2021[["ano", "id_municipio"]]
rais_estudos_2021 = rais_estudos_2021.rename(columns={"id_municipio":"cod_municipio"})
rais_estudos_2021["n_ensino_superior"] = 1
rais_estudos_2021 = rais_estudos_2021.groupby(["cod_municipio", "ano"], as_index=False).sum()
rais_estudos_2021 = rais_estudos_2021.merge(populacao, on="cod_municipio", how="left")
rais_estudos_2021.head()
rais_estudos_2021["n_ensino_superior_por_1k_hab"] = rais_estudos_2021["n_ensino_superior"]*1000/rais_estudos_2021["pop_2021"]

rais_estudos_2021.to_csv(paths_outputs + "rais_ensino_superior.csv", index=False)

###### RAIS Mulheres

In [46]:
path = 'C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/rais_tratada/rais_mulheres/'
rais_mulheres_1 = pd.read_parquet(path+"rais_mulheres000000000037.parquet")
rais_mulheres_2 = pd.read_parquet(path+"rais_mulheres000000000038.parquet")
rais_mulheres_3 = pd.read_parquet(path+"rais_mulheres000000000039.parquet")
rais_mulheres_2021 = pd.concat([rais_mulheres_1, rais_mulheres_2, rais_mulheres_3])

In [47]:
rais_mulheres_2021 = rais_mulheres_2021[["ano", "id_municipio"]]
rais_mulheres_2021 = rais_mulheres_2021.rename(columns={"id_municipio":"cod_municipio"})
rais_mulheres_2021["n_mulheres"] = 1
rais_mulheres_2021 = rais_mulheres_2021.groupby(["cod_municipio", "ano"], as_index=False).sum()
rais_mulheres_2021 = rais_mulheres_2021.merge(populacao, on="cod_municipio", how="left")
rais_mulheres_2021["n_mulheres_por_1k_hab"] = rais_mulheres_2021["n_mulheres"]*1000/rais_mulheres_2021["pop_2021"]

rais_mulheres_2021.to_csv(paths_outputs + "rais_mulheres.csv", index=False)

#### Trabalho Infantil

In [151]:
path_to_cad = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/cad_unico/cad_unico_total.csv"
cad = pd.read_csv(path_to_cad,  encoding="latin1")
cad = cad[["Código", "Referência", "Quantidade total de famílias cadastradas" ]]
cad = cad.rename(columns={"Código":"cod_municipio",
                     "Referência":"ano",
                        "Quantidade total de famílias cadastradas":"familias_cadastradas"})

cad["ano"] =  cad["ano"].apply(lambda x: int(x[-4:]))

cad = cad.query("ano==2023")
cad = cad[["cod_municipio", "familias_cadastradas"]]
cad = cad.groupby("cod_municipio").sum().reset_index()

path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/cad_unico/trabalho_infantil.csv"
base = pd.read_csv(path, encoding="latin1")
base.rename(columns={
    "Código":"cod_municipio",
    'Quantidade total de famílias cadastradas com situação de trabalho infantil':"trabalho_infantil",
    "Referência":"data"}, inplace=True)
base = base[["cod_municipio", "data", "trabalho_infantil"]]
base["data"] = base["data"].apply(lambda x: x.split("/")[1])

C:\Users\gaolg\AppData\Local\Temp\ipykernel_8152\2852476858.py:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  cad = pd.read_csv(path_to_cad,  encoding="latin1")


In [152]:
import math
base = base.query("data == '2021'").groupby(["cod_municipio", "data"], as_index=False).mean()
base["trabalho_infantil"] = base["trabalho_infantil"].apply(lambda x:  math.ceil(x))
base["cod_municipio"] = base["cod_municipio"].astype(str)
cad["cod_municipio"] = cad["cod_municipio"].astype(str)
base  = base.merge(cad, on="cod_municipio", how="left")
base["per_trabalho_infantil"] = base["trabalho_infantil"]*1000/base["familias_cadastradas"]
base.to_csv(paths_outputs + "trabalho_infantil.csv", index=False)

#### Violência

In [104]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/populacao_genero.csv"
pop_genero = pd.read_csv(path)
pop_genero.rename(columns={"Município":"cod_municipio", "pop_2021":"pop_feminina"}, inplace=True)
pop_genero = pop_genero.melt(id_vars=['cod_municipio'], var_name='genero', value_name='populacao')
pop_genero["cod_municipio"] = pop_genero["cod_municipio"].str.extract('(\d+)')
pop_genero.query("genero == 'Feminino'", inplace=True)
pop_genero = pop_genero[["cod_municipio", "populacao"]].reset_index(drop=True)


,cod_municipio,populacao
0,110001,11013
1,110002,55652
2,110003,2486
3,110004,44102
4,110005,8168
...,...,...
5565,522200,7104
5566,522205,4223
5567,522220,2709
5568,522230,2889


In [106]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/populacao_genero.csv"
pop_genero = pd.read_csv(path)
pop_genero.rename(columns={"Município":"cod_municipio", "pop_2021":"pop_feminina"}, inplace=True)
pop_genero = pop_genero.melt(id_vars=['cod_municipio'], var_name='genero', value_name='populacao')
pop_genero["cod_municipio"] = pop_genero["cod_municipio"].str.extract('(\d+)')
pop_genero.query("genero == 'Feminino'", inplace=True)
pop_genero = pop_genero[["cod_municipio", "populacao"]].reset_index(drop=True)

path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Fundamentos para o Bem-estar/violencia/violencia_feminina.csv"
violencia_feminia = pd.read_csv(path)
violencia_feminia_long = pd.melt(violencia_feminia, id_vars=['Município de notificação'], var_name='ano', value_name='n_casos')
violencia_feminia_long = violencia_feminia_long.rename(columns={"Município de notificação":"nome_municipio"})

violencia_feminia_long = violencia_feminia_long.query("ano != 'Total'")
violencia_feminia_long["n_casos"] = violencia_feminia_long["n_casos"].apply(lambda x: int(x.replace("-", "0")))
violencia_feminia_long = violencia_feminia_long.query("nome_municipio != 'Total'")

violencia_feminia_long["cod_municipio"] = violencia_feminia_long["nome_municipio"].str.extract('(\d+)')


violencia_feminia_long = violencia_feminia_long.query("ano == '2021'").reset_index(drop=True)
violencia_feminia_long = violencia_feminia_long[['cod_municipio', 'n_casos']]
violencia_feminia_long = violencia_feminia_long.merge(pop_genero, on="cod_municipio", how="left")

violencia_feminia_long["n_casos_por_1k_hab"] = violencia_feminia_long["n_casos"]*1000/violencia_feminia_long["populacao"]
violencia_feminia_long.to_csv(paths_outputs + "violencia_feminina.csv", index=False)

In [108]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/populacao_idade.csv"
pop_genero = pd.read_csv(path)
pop_genero["0_14"] = pop_genero["0 a 4 anos"] + pop_genero["5 a 9 anos"] + pop_genero["10 a 14 anos"]


,Município,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 29 anos,30 a 39 anos,40 a 49 anos,50 a 59 anos,60 a 69 anos,70 a 79 anos,80 anos e mais,Total
0,110001 ALTA FLORESTA D'OESTE,1601,1592,1556,1623,3652,3568,3101,2901,1782,812,328,22516
1,110002 ARIQUEMES,8396,8262,8317,9490,20100,18822,15160,11695,6809,2991,1106,111148
2,110003 CABIXI,379,375,332,292,724,754,785,699,459,191,77,5067
3,110004 CACOAL,5846,5939,6065,6412,14391,14626,12374,10077,6509,2939,1238,86416
4,110005 CEREJEIRAS,1068,1097,1077,1094,2594,2648,2254,2129,1263,617,247,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522200 VIANOPOLIS,962,970,884,950,2195,2229,2168,1813,1085,566,266,14088
5566,522205 VICENTINOPOLIS,671,680,640,648,1414,1555,1351,1082,588,259,114,9002
5567,522220 VILA BOA,571,544,457,494,1143,1299,924,586,276,117,40,6451
5568,522230 VILA PROPICIO,436,425,342,359,881,849,892,855,539,271,92,5941


In [113]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/populacao_idade.csv"
pop_idade = pd.read_csv(path)
pop_idade["0_14"] = pop_idade["0 a 4 anos"] + pop_idade["5 a 9 anos"] + pop_idade["10 a 14 anos"]
pop_idade.rename(columns={"Município":"cod_municipio", "pop_2021":"pop_feminina"}, inplace=True)
pop_idade = pop_idade.melt(id_vars=['cod_municipio'], var_name='idade', value_name='populacao')
pop_idade["cod_municipio"] = pop_idade["cod_municipio"].str.extract('(\d+)')
pop_idade.query("idade == '0_14'", inplace=True)
pop_idade = pop_idade[["cod_municipio", "populacao"]].reset_index(drop=True)
pop_idade.head()


,cod_municipio,populacao
0,110001,4749
1,110002,24975
2,110003,1086
3,110004,17850
4,110005,3242


In [115]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Fundamentos para o Bem-estar/violencia/violencia_infantil.csv"
violencia_infantil = pd.read_csv(path)
violencia_infantil_long = pd.melt(violencia_infantil, id_vars=['Município de notificação'], var_name='ano', value_name='n_casos')
violencia_infantil_long = violencia_infantil_long.rename(columns={"Município de notificação":"nome_municipio"})

violencia_infantil_long = violencia_infantil_long.query("ano != 'Total'")
violencia_infantil_long["n_casos"] = violencia_infantil_long["n_casos"].apply(lambda x: int(x.replace("-", "0")))
violencia_infantil_long = violencia_infantil_long.query("nome_municipio != 'Total'")

violencia_infantil_long["cod_municipio"] = violencia_infantil_long["nome_municipio"].str.extract('(\d+)')


violencia_infantil_long = violencia_infantil_long.query("ano == '2021'").reset_index(drop=True)
violencia_infantil_long = violencia_infantil_long[['cod_municipio', 'n_casos']]
violencia_infantil_long = violencia_infantil_long.merge(pop_idade, on="cod_municipio", how="left")

violencia_infantil_long["n_casos_por_100k_hab"] = violencia_infantil_long["n_casos"]*100000/violencia_infantil_long["populacao"]
violencia_infantil_long
violencia_infantil_long.to_csv(paths_outputs + "violencia_infantil.csv", index=False)

In [82]:
path = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Fundamentos para o Bem-estar/violencia/violencia_indigenas.csv"
violencia_indigenas = pd.read_csv(path)
violencia_indigenas
violencia_indigenas_long = pd.melt(violencia_indigenas, id_vars=['Município de notificação'], var_name='ano', value_name='n_casos')
violencia_indigenas_long = violencia_indigenas_long.rename(columns={"Município de notificação":"nome_municipio"})

violencia_indigenas_long = violencia_indigenas_long.query("ano != 'Total'")
violencia_indigenas_long["n_casos"] = violencia_indigenas_long["n_casos"].apply(lambda x: int(x.replace("-", "0")))
violencia_indigenas_long = violencia_indigenas_long.query("nome_municipio != 'Total'")

violencia_indigenas_long["cod_municipio"] = violencia_indigenas_long["nome_municipio"].str.extract('(\d+)')


violencia_indigenas_long = violencia_indigenas_long.query("ano == '2021'").reset_index(drop=True)
violencia_indigenas_long = violencia_indigenas_long[['cod_municipio', 'n_casos']]
violencia_indigenas_long = violencia_indigenas_long.merge(censo_populacao_etnia, on="cod_municipio", how="left")

violencia_indigenas_long["n_casos_por_1k_hab"] = violencia_indigenas_long["n_casos"]*1000/violencia_indigenas_long["populacao"]
violencia_indigenas_long.to_csv(paths_outputs + "violencia_indigenas.csv", index=False)


#### Eventos

In [161]:
path  = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Oportunidades/eventos/cultura.csv"
cultura = pd.read_csv(path)

collumns_to_keep = ["Mcul352", "Mcul353", "Mcul355", "Mcul358", "Mcul359", "Mcul3511", "Mcul3901", "Mcul3902", "Mcul3903", "Mcul3904", "Mcul3905", "Mcul3906",
"Mcul3907", "Mcul3909", "Mcul3913", "Mcul3919"]

cultura = cultura[["cod_municipio", "Mcul352", "Mcul353", "Mcul355", "Mcul358", "Mcul359", "Mcul3511", "Mcul3901", "Mcul3902", "Mcul3903", "Mcul3904", "Mcul3905", "Mcul3906",
"Mcul3907", "Mcul3909", "Mcul3913", "Mcul3919"]]

def replacer(x):
    if x == "Sim":
        return 1
    else:
        return 0
for col in collumns_to_keep:
    cultura[col] = cultura[col].apply(lambda x: replacer(x))

cultura["n_eventos"] = cultura[collumns_to_keep].sum(axis=1)
cultura = cultura[["cod_municipio", "n_eventos"]]


In [4]:
def bool_to_int(x):
    if x>0:
        return 1
    else:
        return 0

def replacer(x):
    if x == "Sim":
        return 1
    else:
        return 0
for col in collumns_to_keep:
    cultura[col] = cultura[col].apply(lambda x: replacer(x))

NameError: name 'cultura' is not defined

#### Direitos humanos

In [8]:
import pandas as pd
path  = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Oportunidades/eventos/direitos_humanos.csv"
dir_hum = pd.read_csv(path)

columns_to_clean = [
    "MDHU571", "MDHU572", "MDHU573",
    "MDHU574", "MDHU575", "MDHU576", "MDHU577", "MDHU578",
    "MDHU579", "MDHU571", "MDHU5710", "MDHU5711",
    "MDHU5712","MDHU5713", "MDHU5714", "MDHU5715"]

collumns_to_keep = [
    "cod_municipio", "MDHU571", "MDHU572", "MDHU573",
    "MDHU574", "MDHU575", "MDHU576", "MDHU577", "MDHU578",
    "MDHU579", "MDHU571", "MDHU5710", "MDHU5711",
    "MDHU5712","MDHU5713", "MDHU5714", "MDHU5715"] 

dir_hum1 = dir_hum.copy()
dir_hum2 = dir_hum.copy()

def replacer(x):
    if x == "Sim":
        return 1
    else:
        return 0
for col in columns_to_clean:
    dir_hum1[col] = dir_hum1[col].apply(lambda x: replacer(x))

dir_hum1 = dir_hum1[collumns_to_keep]

dir_hum1["n_dir"] = dir_hum1[columns_to_clean].sum(axis=1)
dir_hum1 = dir_hum1[["cod_municipio", "n_dir"]]

dir_hum1["pelo_menos_um_dir"] = dir_hum1["n_dir"].apply(lambda x: bool_to_int(x))
dir_hum1.to_csv(paths_outputs + "direitos_humanos.csv", index=False)


C:\Users\gaolg\AppData\Local\Temp\ipykernel_8264\4095645537.py:3: DtypeWarning: Columns (341,342,343) have mixed types. Specify dtype option on import or set low_memory=False.
  dir_hum = pd.read_csv(path)


In [10]:
acoes_grupos = dir_hum.copy()

collumns_to_keep = [
    "cod_municipio", "MDHU081", "MDHU082", "MDHU083",
    "MDHU084", "MDHU085", "MDHU086", "MDHU087",
    "MDHU088", "MDHU089", "MDHU0810", "MDHU0811", "MDHU0812"]

collumns_to_clean= [
    "MDHU081", "MDHU082", "MDHU083",
    "MDHU084", "MDHU085", "MDHU086", "MDHU087",
    "MDHU088", "MDHU089", "MDHU0810", "MDHU0811", "MDHU0812"]

for col in collumns_to_clean:
    acoes_grupos[col] = acoes_grupos[col].apply(lambda x: replacer(x))

acoes_grupos = acoes_grupos[collumns_to_keep]

acoes_grupos["n_acoes"] = acoes_grupos[collumns_to_clean].sum(axis=1)
acoes_grupos = acoes_grupos[["cod_municipio", "n_acoes"]]
acoes_grupos["pelo_menos_um_acao"] = acoes_grupos["n_acoes"].apply(lambda x: bool_to_int(x))
acoes_grupos.to_csv(paths_outputs + "acoes_grupos.csv", index=False)
